<a href="https://colab.research.google.com/github/sourcecode369/Kaggle-Kernels/blob/master/Best_Custom_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import time
import os
from tqdm import tqdm 

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, LearningRateScheduler, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Conv2D, SpatialDropout1D, BatchNormalization, MaxPool2D, Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.applications.nasnet import NASNetMobile, preprocess_input
from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow import keras
import imgaug as ia
from imgaug import augmenters as iaa

In [0]:
class DataGenerator(Sequence):
  def __init__(self, images, labels, batch_size=64, image_dimension=(32,32,3), shuffle=False, augment=False):
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.dim = image_dimension
    self.shuffle = shuffle
    self.augment = augment
    self.on_epoch_end()
  
  def __len__(self):
    return int(np.floor(len(self.images)/self.batch_size))
  
  def on_epoch_end(self):
    self.indexes = np.arange(len(self.images))
    if self.shuffle:
      return np.random.shuffle(self.indexes)

  def __getitem__(self, index):
    indexes = self.indexes[index * self.batch_size : (index+1) * self.batch_size]
    labels = self.labels[indexes]
    images = self.images[indexes,:,:,:]        
    if self.augment==True:
      print("Data Augmentation in Progress ..")
      images = self.augmentor(self.images)
      print("Done.")
    images = images/255.0
    return images, labels
  
  def augmentor(self, images):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
        iaa.Fliplr(0.5),  
        iaa.Flipud(0.2),  
        sometimes(iaa.Affine(
          scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
          translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
          rotate=(-10, 10),
          shear=(-5, 5), 
          order=[0, 1],
          cval=(0, 255),
          mode=ia.ALL
        )),
        iaa.SomeOf((0, 5),
                    [sometimes(iaa.Superpixels(p_replace=(0, 1.0),
                                                  n_segments=(20, 200))),
                      iaa.OneOf([
                          iaa.GaussianBlur((0, 1.0)),
                          iaa.AverageBlur(k=(3, 5)),
                          iaa.MedianBlur(k=(3, 5)),
                      ]),
                      iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)),
                      iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
                      iaa.SimplexNoiseAlpha(iaa.OneOf([
                          iaa.EdgeDetect(alpha=(0.5, 1.0)),
                          iaa.DirectedEdgeDetect(alpha=(0.5, 1.0),
                                                direction=(0.0, 1.0)),
                      ])),
                      iaa.AdditiveGaussianNoise(loc=0,
                                                scale=(0.0, 0.01 * 255),
                                                per_channel=0.5),
                      iaa.OneOf([
                          iaa.Dropout((0.01, 0.05), per_channel=0.5),
                          iaa.CoarseDropout((0.01, 0.03),
                                            size_percent=(0.01, 0.02),
                                            per_channel=0.2),
                      ]),
                      iaa.Invert(0.01, per_channel=True),
                      iaa.Add((-2, 2), per_channel=0.5),
                      iaa.AddToHueAndSaturation((-1, 1)),
                      iaa.OneOf([
                          iaa.Multiply((0.9, 1.1), per_channel=0.5),
                          iaa.FrequencyNoiseAlpha(
                              exponent=(-1, 0),
                              first=iaa.Multiply((0.9, 1.1),
                                                per_channel=True),
                              second=iaa.ContrastNormalization(
                                  (0.9, 1.1))
                          )
                      ]),
                      sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5),
                                                          sigma=0.25)),
                      sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
                      sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                    ],
                    random_order=True
                    )
        ],
        random_order=True
    )
    return seq.augment_images(images)

In [0]:
class MNISTModel():
  def __init__(self, image_dimension=(32,32,3),model=1,n_classes=10,version=2, depth=11):
    self.n_classes = n_classes
    self.input_dim = image_dimension 
    self.version = version
    self.depth = depth
    if model==1:
      self.model = self.create_simple()
    else:
      self.model = self.create_model()
    model_type = 'ResNet%dv%d' % (self.depth, self.version)
    print("Model: ", model_type)
  
  def summary(self):
    self.model.summary()

  def create_simple(self):
    input_layer = Flatten((32,32,3))
    x = Dense(768,activation='relu')(x)
    output_layer = Dense(self.n_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer=Adam(lr=0.0005), loss="sparse_categorical_crossentropy", metrics=['acc'])
    return model
  def resnet_layer(self,inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
      conv = Conv2D(num_filters,
                    kernel_size=kernel_size,
                    strides=strides,
                    padding='same',
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(1e-4))

      x = inputs
      if conv_first:
          x = conv(x)
          if batch_normalization:
              x = BatchNormalization()(x)
          if activation is not None:
              x = Activation(activation)(x)
      else:
          if batch_normalization:
              x = BatchNormalization()(x)
          if activation is not None:
              x = Activation(activation)(x)
          x = conv(x)
      return x


  def resnet_v1(self, input_shape, depth, num_classes=10):
      if (depth - 2) % 6 != 0:
          raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
      num_filters = 16
      num_res_blocks = int((depth - 2) / 6)
      inputs = Input(shape=input_shape)
      x = self.resnet_layer(inputs=inputs)
      for stack in range(3):
          for res_block in range(num_res_blocks):
              strides = 1
              if stack > 0 and res_block == 0: 
                  strides = 2  
              y = self.resnet_layer(inputs=x,
                              num_filters=num_filters,
                              strides=strides)
              y = self.resnet_layer(inputs=y,
                              num_filters=num_filters,
                              activation=None)
              if stack > 0 and res_block == 0:
                  x = self.resnet_layer(inputs=x,
                                  num_filters=num_filters,
                                  kernel_size=1,
                                  strides=strides,
                                  activation=None,
                                  batch_normalization=False)
              x = keras.layers.add([x, y])
              x = Activation('relu')(x)
          num_filters *= 2
      x = AveragePooling2D(pool_size=8)(x)
      y = Flatten()(x)
      outputs = Dense(num_classes,
                      activation='softmax',
                      kernel_initializer='he_normal')(y)
      model = Model(inputs=inputs, outputs=outputs)
      return model


  def resnet_v2(self,input_shape, depth, num_classes=10):
      if (depth - 2) % 9 != 0:
          raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
      num_filters_in = 16
      num_res_blocks = int((depth - 2) / 9)

      inputs = Input(shape=input_shape)
      x = self.resnet_layer(inputs=inputs,
                      num_filters=num_filters_in,
                      conv_first=True)
      for stage in range(3):
          for res_block in range(num_res_blocks):
              activation = 'relu'
              batch_normalization = True
              strides = 1
              if stage == 0:
                  num_filters_out = num_filters_in * 4
                  if res_block == 0: 
                      activation = None
                      batch_normalization = False
              else:
                  num_filters_out = num_filters_in * 2
                  if res_block == 0:  
                      strides = 2    

              y = self.resnet_layer(inputs=x,
                              num_filters=num_filters_in,
                              kernel_size=1,
                              strides=strides,
                              activation=activation,
                              batch_normalization=batch_normalization,
                              conv_first=False)
              y = self.resnet_layer(inputs=y,
                              num_filters=num_filters_in,
                              conv_first=False)
              y = self.resnet_layer(inputs=y,
                              num_filters=num_filters_out,
                              kernel_size=1,
                              conv_first=False)
              if res_block == 0:
                  x = self.resnet_layer(inputs=x,
                                  num_filters=num_filters_out,
                                  kernel_size=1,
                                  strides=strides,
                                  activation=None,
                                  batch_normalization=False)
              x = keras.layers.add([x, y])

          num_filters_in = num_filters_out

      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = AveragePooling2D(pool_size=8)(x)
      y = Flatten()(x)
      outputs = Dense(num_classes,
                      activation='softmax',
                      kernel_initializer='he_normal')(y)

      model = Model(inputs=inputs, outputs=outputs)
      return model

  def create_model(self):
    if self.version == 2:
        model = self.resnet_v2(input_shape=self.input_dim, depth=self.depth, num_classes=self.n_classes)
    else:
        model = self.resnet_v1(input_shape=self.input_dim, depth=self.depth, num_classes=self.n_classes)
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=self.lr_schedule(0)),
              metrics=['accuracy'])
    return model

  def lr_schedule(self, epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
    
  def train(self, train_data, val_data, plot_results=True):
    print("Starting Training")
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                                patience=2,
                                                verbose=1,
                                                factor=0.5,
                                                min_lr=0.000000001)
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=5,
                                   mode="min",
                                   restore_best_weights=True,
                                   verbose=1)
    
    checkpoint = ModelCheckpoint('./model_weights.hdf5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='min',
                                 save_weights_only=True)
    
    history = self.model.fit_generator(generator=train_data,
		                                   validation_data=val_data,
		                                   epochs=EPOCHS,
		                                   steps_per_epoch=len(train_data),
		                                   validation_steps =len(val_data),
		                                   callbacks=[learning_rate_reduction, early_stopping, checkpoint],
		                                   verbose=1,
		                                   )
    if plot_results:
      fig, ax = plt.subplots(2, 1, figsize=(6, 6))
      ax[0].plot(history.history['loss'], label="TrainLoss")
      ax[0].plot(history.history['val_loss'], label="ValLoss")
      ax[0].legend(loc='best', shadow=True)

      ax[1].plot(history.history['acc'], label="TrainAcc")
      ax[1].plot(history.history['val_acc'], label="ValAcc")
      ax[1].legend(loc='best', shadow=True)
      plt.show()
   
  def create_submit(self, test_data):
    'Create basic file submit'
    self.model.load_weights("./model_weights.hdf5")
    results = self.model.predict_generator(test_data)
    results_to_save = pd.DataFrame({"id": test_data.images_paths,
                                    "label": results[:,0]
                                    })

    results_to_save["id"] = results_to_save["id"].apply(lambda x: x.replace("../input/test/", "").replace(".tif", ""))
    results_to_save.to_csv("./submission.csv", index=False)

In [0]:
from sklearn.model_selection import train_test_split
def loadData(db, val_split=0.2,sub_sample_size=-1):
  (X_train, y_train), (X_test,y_test) = cifar10.load_data()
  if db == "train":
    if val_split>0:
      X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=val_split,stratify=y_train)
      print("Train Data completed")
      train_data = DataGenerator(X_train, y_train, batch_size=BATCH_SIZE,image_dimension=(32,32,3), shuffle=True, augment=False)
      print("val Data completed")
      val_data = DataGenerator(X_val, y_val, batch_size=BATCH_SIZE,image_dimension=(32,32,3), shuffle=False, augment=False)
      return train_data, val_data
    else:
      return DataGenerator(X_train,y_train,batch_size=BATCH_SIZE,image_dimension=(32,32,3),augment=True,shuffle=True), None  
  else:
    if sub_sample_size== -1 :
      return DataGenerator(X_test, y_test, batch_size=2), None
    else:
      return DataGenerator(X_test[:sub_sample_size], y_test[:sub_sample_size], batch_size=1), None

In [0]:
if __name__ == "__main__":
	EPOCHS = 10
	BATCH_SIZE = 64
	IMAGE_DIMENSIONS = (32,32 ,1)

	model = MNISTModel(IMAGE_DIMENSIONS,model=1,n_classes=10,version=2,depth=11)
	model.summary()

	# train model
	train_data, val_data = loadData("train", val_split=0.2)
	model.train(train_data, val_data, plot_results=True)

	# submit model
	test_data, _ = loadData("test")
	model.create_submit(test_data)

In [0]:
# https://www.kaggle.com/c/quickdraw-doodle-recognition/discussion/71760
# https://www.kaggle.com/rejpalcz/cnn-128x128x4-keras-from-scratch-lb-0-328
# https://www.kaggle.com/mpalermo/keras-pipeline-custom-generator-imgaug
# https://www.kaggle.com/c/human-protein-atlas-image-classification/discussion/73929
# https://www.kaggle.com/fergusoci/keras-loss-based-learning-rate-scheduler
# https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/80807
# https://www.kaggle.com/lavanyashukla01/better-models-faster-with-weights-biases
# https://www.kaggle.com/kozistr/0-99757-lb-top-4-with-keras
# https://www.kaggle.com/gpiosenka/tf2-keras-cnn-with-custom-callbacks
# https://www.kaggle.com/ashhafez/keras-custom-learning-rate-decay
# https://www.kaggle.com/danmoller/make-best-use-of-a-kernel-s-limited-uptime-keras
# https://www.kaggle.com/residentmario/keras-callbacks-and-config-files
# https://github.com/analyticsindiamagazine/MachineHack/blob/master/Hackathon_Solutions/Chartbusters%20Prediction%20:%20Foretell%20The%20Popularity%20Of%20Songs/Rank_1_Nikhil/best_model.ipynb